In [15]:
import requests
import json

def get_departures(stop_id):
    """Abfahrtszeiten für eine bestimmte Haltestelle abrufen"""
    url = f"http://www.wienerlinien.at/ogd_realtime/monitor?stopId={stop_id}"
    
    response = requests.get(url)
    data = response.json()
    
    if "data" in data and "monitors" in data["data"]:
        return data["data"]["monitors"]
    return None

def print_departures(stop_id):
    """Abfahrtszeiten formatiert ausgeben"""
    departures = get_departures(stop_id)
    
    if not departures:
        print("Keine Daten gefunden")
        return
    
    countdown_list = []
    countdown_strings = []

    for monitor in departures:
        location = monitor["locationStop"]["properties"]["title"]
        print(f"Haltestelle: {location}")
        print("-" * 40)
        line_data_full = []

        for line in monitor["lines"]:
            line_data_full.append(line)
            line_name = line["name"]
            towards = line["towards"]
            print(f"Linie: {line_name} Richtung: {towards}")
            
            if "departures" in line and "departure" in line["departures"]:
                for departure in line["departures"]["departure"]:
                    countdown = departure["departureTime"]["countdown"]
                    print(f"  Abfahrt in {countdown} Minuten")
                    countdown_list.append(countdown)
                    countdown_strings.append(f'Abfahrt in {countdown} Minuten')
            print()
    return countdown, countdown_strings, line_data_full
# Beispiel für die Haltestelle Stephansplatz (ID: 4210)
countdown,countdown_strings, line_data_full = print_departures(4210)


Haltestelle: Rathaus
----------------------------------------
Linie: U2 Richtung: SEESTADT         
  Abfahrt in 3 Minuten
  Abfahrt in 10 Minuten
  Abfahrt in 17 Minuten
  Abfahrt in 24 Minuten
  Abfahrt in 32 Minuten
  Abfahrt in 39 Minuten
  Abfahrt in 47 Minuten
  Abfahrt in 54 Minuten
  Abfahrt in 62 Minuten
  Abfahrt in 69 Minuten



In [13]:
countdown_strings

['Abfahrt in 4 Minuten',
 'Abfahrt in 11 Minuten',
 'Abfahrt in 17 Minuten',
 'Abfahrt in 25 Minuten',
 'Abfahrt in 32 Minuten',
 'Abfahrt in 40 Minuten',
 'Abfahrt in 47 Minuten',
 'Abfahrt in 55 Minuten',
 'Abfahrt in 62 Minuten']